<a href="https://colab.research.google.com/github/piyush888/AnimeRecommendation/blob/main/anime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keybert
!pip install keyphrase-vectorizers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.3 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=09c6ec230bba5cbde8e34329dec67e229b91fe7ae8404bd55a760059ff820129
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1caf6a5d545007137bd3f28e6601a7dd388b84d44b70d9ccc9c22ca7b0fad68a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 10.4 MB/s eta 0:00:0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

csv_path = "/content/drive/MyDrive/Anime/dataset/animes.csv"
data_df = pd.read_csv(csv_path)
data_df.head(5)

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


In [ ]:
data_df.columns

Index(['uid', 'title', 'synopsis', 'genre', 'aired', 'episodes', 'members',
       'popularity', 'ranked', 'score', 'img_url', 'link'],
      dtype='object')

In [ ]:
original_df = data_df.copy()

data_df = data_df[['title', 'synopsis', 'genre', 'aired', 'popularity', 'score']]
data_df.head()

,title,synopsis,genre,aired,popularity,score
0,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",141,8.82
1,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",28,8.83
2,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",98,8.83
3,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",4,9.23
4,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",502,8.83


In [ ]:
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT

# Init German KeyBERT model
kw_model = KeyBERT()

# Init vectorizer for the German language
vectorizer = KeyphraseCountVectorizer()

# # Get German keyphrases
# kw_model.extract_keywords(docs=german_docs, vectorizer=vectorizer)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
data_df = data_df.dropna(subset='synopsis')
data_df['synopsis']

0        Following their participation at the Inter-Hig...
1        Music accompanies the path of the human metron...
2        The Abyss—a gaping chasm stretching down into ...
3        "In order for something to be obtained, someth...
4        After helping revive the legendary vampire Kis...
                               ...                        
19306    Cocona is an average middle schooler living wi...
19307    While visiting the National Library, junior-hi...
19308    Years ago, all of the ghosts in a haunted scho...
19309    Inuyasha and company have finally destroyed Na...
19310    The year is Universal Century 0093. Char Aznab...
Name: synopsis, Length: 18336, dtype: object

In [ ]:
data_df.reset_index(drop=True, inplace=True)
data_df['synopsis']

0        Following their participation at the Inter-Hig...
1        Music accompanies the path of the human metron...
2        The Abyss—a gaping chasm stretching down into ...
3        "In order for something to be obtained, someth...
4        After helping revive the legendary vampire Kis...
                               ...                        
18331    Cocona is an average middle schooler living wi...
18332    While visiting the National Library, junior-hi...
18333    Years ago, all of the ghosts in a haunted scho...
18334    Inuyasha and company have finally destroyed Na...
18335    The year is Universal Century 0093. Char Aznab...
Name: synopsis, Length: 18336, dtype: object

In [ ]:
keywords = kw_model.extract_keywords(docs=data_df['synopsis'], vectorizer=vectorizer, keyphrase_ngram_range=(1,4))

In [ ]:
len(keywords)

18336

In [ ]:
data_df['keywords'] = keywords

In [ ]:
data_df.head()

,title,synopsis,genre,aired,popularity,score,keywords
0,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",141,8.82,"[(karasuno high school volleyball team, 0.7349..."
1,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",28,8.83,"[(prodigious pianist kousei arima, 0.5814), (k..."
2,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",98,8.83,"[(abyss, 0.559), (depths, 0.4324), (novice red..."
3,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",4,9.23,"[(alchemist, 0.5496), (fullmetal alchemist, 0...."
4,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",502,8.83,"[(araragi, 0.4571), (legendary vampire kiss, 0..."


In [ ]:
data_df.to_csv("/content/drive/MyDrive/Anime/dataset/animes_with_keywords.csv")

In [ ]:
data_df = pd.read_csv("/content/drive/MyDrive/Anime/dataset/animes_with_keywords.csv")

In [ ]:
data_df.head()

,Unnamed: 0,title,synopsis,genre,aired,popularity,score,keywords
0,0,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",141,8.82,"[('karasuno high school volleyball team', 0.73..."
1,1,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",28,8.83,"[('prodigious pianist kousei arima', 0.5814), ..."
2,2,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",98,8.83,"[('abyss', 0.559), ('depths', 0.4324), ('novic..."
3,3,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",4,9.23,"[('alchemist', 0.5496), ('fullmetal alchemist'..."
4,4,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",502,8.83,"[('araragi', 0.4571), ('legendary vampire kiss..."


In [ ]:
review_df = pd.read_csv("/content/drive/MyDrive/Anime/dataset/reviews.csv")

In [ ]:
review_df.head(5)

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149


In [ ]:
review_groupby = review_df.groupby(by="anime_uid").agg(list)
review_groupby

,uid,profile,text,score,scores,link
anime_uid,,,,,,
1,"[5740, 48839, 296203, 184551, 159517, 189179, ...","[RangFlash, reinis-jan, maxeraturee, Sephiroth...",[\n \n \n \n ...,"[10, 9, 10, 8, 10, 10, 10, 7, 10, 9, 10, 9, 8,...","[{'Overall': '10', 'Story': '8', 'Animation': ...","[https://myanimelist.net/reviews.php?id=5740, ..."
5,"[315104, 14130, 146405, 248504, 225964, 107669...","[ToastMyGhost, deralte, jyagan0, Phylo606, Wol...",[\n \n \n \n ...,"[8, 8, 7, 5, 9, 8, 10, 8, 9, 7, 10, 10, 6, 8, ...","[{'Overall': '8', 'Story': '7', 'Animation': '...",[https://myanimelist.net/reviews.php?id=315104...
6,"[297815, 183824, 8560, 20262, 221888, 291308, ...","[AxiomOpus, DanteMustDie8907, OkazukiIchigo, c...",[\n \n \n \n ...,"[9, 8, 10, 9, 9, 10, 6, 10, 10, 7, 9, 9, 8, 7,...","[{'Overall': '9', 'Story': '9', 'Animation': '...",[https://myanimelist.net/reviews.php?id=297815...
7,"[149507, 266383, 276875, 215743, 245248, 29872...","[KD7BWB, kayy0c4, Gameboon, metrikfire, Raggad...",[\n \n \n \n ...,"[10, 5, 8, 8, 8, 7, 9, 5, 10, 8, 9, 9, 8, 6, 7...","[{'Overall': '10', 'Story': '10', 'Animation':...",[https://myanimelist.net/reviews.php?id=149507...
8,"[255242, 150985, 22059, 17881]","[DesolatePsyche, ali12345456, Uthred, shintai88]",[\n \n \n \n ...,"[7, 7, 6, 8]","[{'Overall': '7', 'Story': '7', 'Animation': '...",[https://myanimelist.net/reviews.php?id=255242...
...,...,...,...,...,...,...
40489,"[320843, 319234, 319205, 319334, 319210]","[Rascaz, Inorichi, Caspias, NiQ1, Electrocute4u]",[\n \n \n \n ...,"[8, 6, 9, 7, 9]","[{'Overall': '8', 'Story': '0', 'Animation': '...",[https://myanimelist.net/reviews.php?id=320843...
40542,"[325439, 325620, 325350, 325323, 325341, 32533...","[LeonardoXD, AlecMAL, rea10969, animejas, The-...",[\n \n \n \n ...,"[8, 8, 10, 7, 7, 10, 8, 8, 10, 7, 7, 10]","[{'Overall': '8', 'Story': '0', 'Animation': '...",[https://myanimelist.net/reviews.php?id=325439...
40693,[325253],[8mann],[\n \n \n \n ...,[7],"[{'Overall': '7', 'Story': '5', 'Animation': '...",[https://myanimelist.net/reviews.php?id=325253]


In [ ]:
review_groupby.to_csv("/content/drive/MyDrive/Anime/dataset/review_groupby.csv")

In [ ]:
review_groupby = pd.read_csv("/content/drive/MyDrive/Anime/dataset/review_groupby.csv")
review_groupby.head(5)

,anime_uid,uid,profile,text,score,scores,link
0,1,"[5740, 48839, 296203, 184551, 159517, 189179, ...","['RangFlash', 'reinis-jan', 'maxeraturee', 'Se...","[""\n \n \n \n ...","[10, 9, 10, 8, 10, 10, 10, 7, 10, 9, 10, 9, 8,...","[""{'Overall': '10', 'Story': '8', 'Animation':...",['https://myanimelist.net/reviews.php?id=5740'...
1,5,"[315104, 14130, 146405, 248504, 225964, 107669...","['ToastMyGhost', 'deralte', 'jyagan0', 'Phylo6...","[""\n \n \n \n ...","[8, 8, 7, 5, 9, 8, 10, 8, 9, 7, 10, 10, 6, 8, ...","[""{'Overall': '8', 'Story': '7', 'Animation': ...",['https://myanimelist.net/reviews.php?id=31510...
2,6,"[297815, 183824, 8560, 20262, 221888, 291308, ...","['AxiomOpus', 'DanteMustDie8907', 'OkazukiIchi...",['\n \n \n \n ...,"[9, 8, 10, 9, 9, 10, 6, 10, 10, 7, 9, 9, 8, 7,...","[""{'Overall': '9', 'Story': '9', 'Animation': ...",['https://myanimelist.net/reviews.php?id=29781...
3,7,"[149507, 266383, 276875, 215743, 245248, 29872...","['KD7BWB', 'kayy0c4', 'Gameboon', 'metrikfire'...",['\n \n \n \n ...,"[10, 5, 8, 8, 8, 7, 9, 5, 10, 8, 9, 9, 8, 6, 7...","[""{'Overall': '10', 'Story': '10', 'Animation'...",['https://myanimelist.net/reviews.php?id=14950...
4,8,"[255242, 150985, 22059, 17881]","['DesolatePsyche', 'ali12345456', 'Uthred', 's...",['\n \n \n \n ...,"[7, 7, 6, 8]","[""{'Overall': '7', 'Story': '7', 'Animation': ...",['https://myanimelist.net/reviews.php?id=25524...


In [ ]:
review_groupby.columns

Index(['anime_uid', 'uid', 'profile', 'text', 'score', 'scores', 'link'], dtype='object')

In [ ]:
review_groupby = review_groupby[['anime_uid', 'uid', 'profile', 'text', 'score', 'scores']]

In [ ]:
reviews_count = {}
for review in review_groupby.iterrows():
  print(review[1].text)
  print(type(review[1].text), len(review[1].text))
  reviews_count[review[1].anime_uid] = len(review[1]['text'])
  break
print(reviews_count)

["\n           \n         \n           \n             \n           \n         \n         \n           more pics \n         \n       \n         \n       \n         \n           Overall \n           10 \n         \n         \n           Story \n           8 \n         \n                   \n             Animation \n             9 \n           \n           \n             Sound \n             10 \n           \n                 \n           Character \n           10 \n         \n         \n           Enjoyment \n           10 \n         \n       \n     \n\n                    \n    Cowboy Bebop. Perhaps the greatest anime TV show created thus far. It's a massive mixture of genres, but is mostly sci-fi, comedy and action with a western vibe and a touch of film noir. \r\n \r\nThe story of Cowboy Bebop, or the lack thereof, is probably the weakest link to the series. This series does not have an ongoing story. However, the story is NOT the main focus of the show. Each episode can stand on its 

In [ ]:
list_of_reviews_list = (review_groupby.iloc[i:i+1000] for i in range(0, len(review_groupby), 1000))


In [ ]:
first_1000 = next(list_of_reviews_list)

In [ ]:
for review in first_1000.iterrows():
  print(review[1]['anime_uid'])
  break

1


In [ ]:
first_1000_reviews_list = [review[1]['text'] for review in first_1000.iterrows()]

In [ ]:
with open("first1000.txt", "w") as f:
  f.writelines(first_1000_reviews_list)